# ROC-Kurve - Manuelle Berechnung

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_12/roc_kurve_manuell.ipynb)

Dieses Notebook zeigt, wie eine ROC-Kurve aus expliziten Werten berechnet wird, ohne sklearn oder andere Bibliotheken zu verwenden.

**Gegeben:**
- 10 Samples: 5 OK (negativ) und 5 NOK (positiv)
- Defektwahrscheinlichkeiten so gewählt, dass bei Schwelle 0.5:
  - 2 False Positive (OK als NOK klassifiziert)
  - 1 False Negative (NOK als OK klassifiziert)

In [ ]:
# Nur grundlegende Bibliotheken
import matplotlib.pyplot as plt
import numpy as np

## 1. Daten definieren

Wir definieren 10 Samples mit ihren wahren Labels und vorhergesagten Defektwahrscheinlichkeiten.

In [ ]:
# Wahre Labels: 0 = OK (negativ), 1 = NOK (positiv)
true_labels = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

# Defektwahrscheinlichkeiten so gewählt, dass bei Schwelle 0.5:
# - 2 False Positive: OK-Samples mit Wahrscheinlichkeit > 0.5
# - 1 False Negative: NOK-Sample mit Wahrscheinlichkeit < 0.5
predicted_probs = [
    0.2,   # OK - richtig klassifiziert
    0.3,   # OK - richtig klassifiziert  
    0.1,   # OK - richtig klassifiziert
    0.6,   # OK - False Positive (FP)
    0.7,   # OK - False Positive (FP)
    0.8,   # NOK - richtig klassifiziert
    0.9,   # NOK - richtig klassifiziert
    0.4,   # NOK - False Negative (FN)
    0.85,  # NOK - richtig klassifiziert
    0.75   # NOK - richtig klassifiziert
]

print("Sample\tWahr\tWahrsch.\tBei Schwelle 0.5")
print("-" * 40)
for i in range(len(true_labels)):
    pred_class = 1 if predicted_probs[i] >= 0.5 else 0
    result = "TP" if true_labels[i] == 1 and pred_class == 1 else \
             "TN" if true_labels[i] == 0 and pred_class == 0 else \
             "FP" if true_labels[i] == 0 and pred_class == 1 else "FN"
    print(f"{i+1}\t{true_labels[i]}\t{predicted_probs[i]:.2f}\t\t{result}")

# Verifikation der Bedingungen
fp_count = sum(1 for i in range(len(true_labels)) 
               if true_labels[i] == 0 and predicted_probs[i] >= 0.5)
fn_count = sum(1 for i in range(len(true_labels)) 
               if true_labels[i] == 1 and predicted_probs[i] < 0.5)

print(f"\nBei Schwelle 0.5: {fp_count} False Positive, {fn_count} False Negative")

## 2. Funktion zur Berechnung von TPR und FPR

**True Positive Rate (TPR)** = Sensitivität = TP / (TP + FN)  
**False Positive Rate (FPR)** = 1 - Spezifität = FP / (FP + TN)

In [ ]:
def calculate_tpr_fpr(true_labels, predicted_probs, threshold):
    """
    Berechnet TPR und FPR für eine gegebene Schwelle.
    
    Parameters:
    - true_labels: Liste der wahren Labels (0/1)
    - predicted_probs: Liste der vorhergesagten Wahrscheinlichkeiten
    - threshold: Schwellenwert für Klassifikation
    
    Returns:
    - tpr: True Positive Rate
    - fpr: False Positive Rate
    """
    
    # Vorhersagen basierend auf Schwelle
    predictions = [1 if prob >= threshold else 0 for prob in predicted_probs]
    
    # Konfusionsmatrix berechnen
    tp = sum(1 for i in range(len(true_labels)) 
             if true_labels[i] == 1 and predictions[i] == 1)
    tn = sum(1 for i in range(len(true_labels)) 
             if true_labels[i] == 0 and predictions[i] == 0)
    fp = sum(1 for i in range(len(true_labels)) 
             if true_labels[i] == 0 and predictions[i] == 1)
    fn = sum(1 for i in range(len(true_labels)) 
             if true_labels[i] == 1 and predictions[i] == 0)
    
    # TPR und FPR berechnen
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    return tpr, fpr, tp, tn, fp, fn

# Test der Funktion mit Schwelle 0.5
tpr, fpr, tp, tn, fp, fn = calculate_tpr_fpr(true_labels, predicted_probs, 0.5)

print(f"Bei Schwelle 0.5:")
print(f"TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")
print(f"TPR (Sensitivität): {tpr:.3f}")
print(f"FPR (1 - Spezifität): {fpr:.3f}")

## 3. ROC-Kurve berechnen

Wir berechnen TPR und FPR für verschiedene Schwellenwerte und erstellen die ROC-Kurve.

In [ ]:
# Schwellenwerte definieren
# Wir nehmen alle einzigartigen Wahrscheinlichkeiten plus 0 und 1
unique_probs = sorted(set(predicted_probs))
thresholds = [0.0] + unique_probs + [1.0]
thresholds = sorted(set(thresholds), reverse=True)  # Absteigende Reihenfolge

print("Schwellenwerte:", [f"{t:.2f}" for t in thresholds])

# TPR und FPR für alle Schwellenwerte berechnen
tpr_values = []
fpr_values = []

print("\nSchwelle\tTPR\tFPR\tTP\tTN\tFP\tFN")
print("-" * 50)

for threshold in thresholds:
    tpr, fpr, tp, tn, fp, fn = calculate_tpr_fpr(true_labels, predicted_probs, threshold)
    tpr_values.append(tpr)
    fpr_values.append(fpr)
    print(f"{threshold:.2f}\t\t{tpr:.3f}\t{fpr:.3f}\t{tp}\t{tn}\t{fp}\t{fn}")

print(f"\nROC-Punkte (FPR, TPR):")
for i, threshold in enumerate(thresholds):
    print(f"Schwelle {threshold:.2f}: ({fpr_values[i]:.3f}, {tpr_values[i]:.3f})")

## 4. ROC-Kurve visualisieren

In [ ]:
# ROC-Kurve plotten
plt.figure(figsize=(10, 8))

# ROC-Kurve
plt.plot(fpr_values, tpr_values, 'bo-', linewidth=2, markersize=8, label='ROC-Kurve')

# Diagonale (zufällige Klassifikation)
plt.plot([0, 1], [0, 1], 'r--', label='Zufällige Klassifikation (AUC = 0.5)')

# Punkte beschriften
for i, threshold in enumerate(thresholds):
    plt.annotate(f'{threshold:.2f}', 
                (fpr_values[i], tpr_values[i]), 
                xytext=(5, 5), textcoords='offset points',
                fontsize=9, alpha=0.7)

# Spezielle Punkte hervorheben
# Schwelle 0.5 finden
idx_05 = thresholds.index(0.5) if 0.5 in thresholds else None
if idx_05 is not None:
    plt.plot(fpr_values[idx_05], tpr_values[idx_05], 'rs', markersize=12, 
             label=f'Schwelle 0.5 (FPR={fpr_values[idx_05]:.2f}, TPR={tpr_values[idx_05]:.2f})')

plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC-Kurve - Manuelle Berechnung\n10 Samples: 5 OK, 5 NOK')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(-0.05, 1.05)
plt.ylim(-0.05, 1.05)

# AUC approximieren (Trapezregel)
auc = 0
for i in range(len(fpr_values) - 1):
    # Sortierung sicherstellen für AUC-Berechnung
    if fpr_values[i] != fpr_values[i+1]:
        auc += (fpr_values[i+1] - fpr_values[i]) * (tpr_values[i] + tpr_values[i+1]) / 2

# AUC korrigieren (da unsere Listen in umgekehrter Reihenfolge sind)
fpr_sorted = sorted(zip(fpr_values, tpr_values))
auc = 0
for i in range(len(fpr_sorted) - 1):
    x1, y1 = fpr_sorted[i]
    x2, y2 = fpr_sorted[i+1]
    auc += (x2 - x1) * (y1 + y2) / 2

plt.text(0.6, 0.2, f'AUC ≈ {auc:.3f}', fontsize=12, 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

## 5. Zusammenfassung

**Was wir gelernt haben:**

1. **ROC-Kurve entsteht** durch Variation der Klassifikationsschwelle
2. **Jeder Punkt** auf der Kurve entspricht einem TPR/FPR-Paar bei einer bestimmten Schwelle
3. **Perfekter Klassifikator** würde durch Punkt (0,1) gehen
4. **Zufällige Klassifikation** entspricht der Diagonalen
5. **AUC (Area Under Curve)** misst die Gesamtleistung des Klassifikators

**Unsere Daten bei Schwelle 0.5:**
- 2 False Positive (OK als NOK klassifiziert)
- 1 False Negative (NOK als OK klassifiziert)
- TPR = 4/5 = 0.8 (80% der NOK-Fälle erkannt)
- FPR = 2/5 = 0.4 (40% der OK-Fälle fälschlicherweise als NOK klassifiziert)